In [1]:
pip install opencv-python numpy ultralytics torch torchvision tensorflow deep_sort_realtime flask

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 37.3 MB 10.3 MB/s eta 0:00:01
     |████████████████████████████████| 5.3 MB 16.3 MB/s eta 0:00:01
     |████████████████████████████████| 914 kB 23.0 MB/s eta 0:00:01
     |████████████████████████████████| 66.5 MB 12.4 MB/s eta 0:00:01
     |████████████████████████████████| 1.8 MB 12.2 MB/s eta 0:00:01
     |████████████████████████████████| 239.4 MB 10.4 MB/s eta 0:00:01
     |████████████████████████████████| 8.4 MB 10.7 MB/s eta 0:00:01
     |████████████████████████████████| 102 kB 28.5 MB/s eta 0:00:01
     |████████████████████████████████| 78 kB 13.6 MB/s eta 0:00:01
     |████████████████████████████████| 64 kB 14.7 MB/s eta 0:00:01
     |████████████████████████████████| 3.1 MB 11.9 MB/s eta 0:00:01
     |████████████████████████████████| 11.3 MB 35.6 MB/s eta 0:00:01
     |████████████████████████████████| 294 kB 27.6 MB/s eta 0:00:01
     |████████████████

In [2]:
from ultralytics import YOLO
import cv2

# Load YOLOv8 model
model = YOLO("yolov8n.pt")
cap = cv2.VideoCapture("traffic_video.mp4")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)
    for r in results:
        for box in r.boxes.xyxy:
            x1, y1, x2, y2 = map(int, box)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

    cv2.imshow("Traffic Detection", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Matplotlib is building the font cache; this may take a moment.
/Users/mohdmakki/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/Users/mohdmakki/Library/Application Support/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.25M/6.25M [00:01<00:00, 5.14MB/s]
OpenCV: Couldn't read video stream from file "traffic_video.mp4"


In [3]:
from deep_sort_realtime.deepsort_tracker import DeepSort

tracker = DeepSort(max_age=30)

def track_vehicles(frame, results):
    detections = []
    for r in results:
        for box, conf in zip(r.boxes.xyxy, r.boxes.conf):
            x1, y1, x2, y2 = map(int, box)
            detections.append(([x1, y1, x2, y2], conf, "vehicle"))

    tracks = tracker.update_tracks(detections, frame=frame)
    for track in tracks:
        if track.is_confirmed():
            x1, y1, x2, y2 = map(int, track.to_tlbr())
            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)

    return frame


In [9]:
from deep_sort_realtime.deepsort_tracker import DeepSort
import numpy as np

# Initialize the DeepSort tracker
tracker = DeepSort(max_age=30)

def track_vehicles(frame, results):
    detections = []
    
    # Extract bounding boxes and confidence scores
    for r in results:
        for box, conf in zip(r.boxes.xyxy, r.boxes.conf):
            x1, y1, x2, y2 = map(int, box)
            detections.append((np.array([x1, y1, x2, y2]), conf, "vehicle"))  # Ensure correct format
    
    # Update the tracker with detections
    tracks = tracker.update_tracks(detections, frame=frame)

    vehicle_count = 0  # Count confirmed tracks

    for track in tracks:
        if track.is_confirmed():  
            vehicle_count += 1
            x1, y1, x2, y2 = map(int, track.to_tlbr())  # Get bounding box
            track_id = track.track_id  # Unique vehicle ID
            
            # Draw bounding box and ID on the frame
            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
            cv2.putText(frame, f"ID {track_id}", (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    return frame, vehicle_count

# Example Usage:
# results = model(frame)  # YOLO Detection
# frame, count = track_vehicles(frame, results)
# print(f"Vehicles Detected: {count}")


In [10]:
from flask import Flask, render_template
from flask_socketio import SocketIO

app = Flask(__name__)
socketio = SocketIO(app)

@app.route("/")
def index():
    return render_template("index.html")

def send_data():
    while True:
        socketio.emit("update", {"vehicles": vehicle_count, "status": traffic_status})
        socketio.sleep(1)

socketio.start_background_task(send_data)

if __name__ == "__main__":
    socketio.run(app, debug=True)


ModuleNotFoundError: No module named 'flask_socketio'